In [15]:
from pathlib import Path
import pandas as pd
import os
import csv
import datetime as dt


DATA_DIR = Path('DATA')
dfile = DATA_DIR / 'Tuesday-WorkingHours.pcap_ISCX.csv'


#df = pd.concat((pd.read_csv(DATA_DIR/f) for f in os.listdir(DATA_DIR) if f.startswith('Friday')), ignore_index=True)
#dfile = DATA_DIR / 'Friday-WorkingHours-Morning.pcap_ISCX.csv'
def dateparse(date):
    return dt.datetime.strptime(date, '%d/%m/%Y %H:%M')


df = pd.read_csv(dfile, parse_dates=[' Timestamp'], date_parser=dateparse)

df = df.rename(columns=lambda x: x.strip())
df = df[df['Label'] != 'BENIGN']
# df = df[(df['Timestamp']> dt.datetime(2017,7,4,9)) & (df['Timestamp'] < dt.datetime(2017,7,4,10))]
print(df['Label'].unique())

['FTP-Patator' 'SSH-Patator']


In [16]:
QUANT_COLS = ['Flow Duration',
              'Total Fwd Packets', 'Total Backward Packets',
              'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
              'Fwd Packet Length Max', 'Fwd Packet Length Min',
              'Fwd Packet Length Mean', 'Fwd Packet Length Std',
              'Bwd Packet Length Max', 'Bwd Packet Length Min',
              'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
              'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
              'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
              'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
              'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
              'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
              'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
              'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
              'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count',
              'PSH Flag Count', 'ACK Flag Count',
              'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
              'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
              'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
              'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
              'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
              'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean',
              'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
              'Idle Std', 'Idle Max', 'Idle Min', 'Label']

In [17]:
import numpy as np

df = df[QUANT_COLS]
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df[df.select_dtypes(include=[np.number]).ge(0).all(1)]
df = df.dropna()
#df = df[df['Label'] != 'BENIGN']
M = df.to_numpy()
Labels_tmp = M[:, -1]
Labels = Labels_tmp
# Labels = [l if l == 'BENIGN' else 'ATTACK' for l in Labels_tmp ]

M = M[:, :-1]

In [4]:
from sklearn import svm
from sklearn.model_selection import train_test_split

M_train, M_test, Labels_train, Labels_test = train_test_split(M, Labels, test_size=0.4, random_state=0)
#clf = svm.SVC(kernel='linear', C=1).fit(M_train, Labels_train)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

M_train, M_test, Labels_train, Labels_test = train_test_split(M, Labels, test_size=0.4, random_state=0)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(M_train, Labels_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [6]:

clf.score(M_test, Labels_test)

0.9775908161660638

In [18]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

sel = SelectFromModel(RandomForestClassifier(max_depth=2, n_estimators = 100))
sel.fit(M, Labels)

SelectFromModel(estimator=RandomForestClassifier(max_depth=2))

In [19]:
sel.get_support()
selected_feat= np.array(QUANT_COLS[:-1])[(sel.get_support())]
print(selected_feat)

['Total Fwd Packets' 'Total Length of Fwd Packets' 'Fwd Packet Length Max'
 'Fwd Packet Length Mean' 'Fwd Packet Length Std' 'Bwd Packet Length Mean'
 'Bwd Packet Length Std' 'Flow Bytes/s' 'Fwd IAT Total' 'Fwd IAT Max'
 'Fwd Header Length' 'Bwd Packets/s' 'Max Packet Length'
 'Packet Length Mean' 'Packet Length Std' 'Packet Length Variance'
 'Average Packet Size' 'Avg Fwd Segment Size' 'Fwd Header Length.1'
 'Subflow Fwd Packets' 'Subflow Fwd Bytes' 'Subflow Bwd Bytes']


In [20]:
df

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
11347,5216127,3,1,0,0.0,0,0,0.000000,0.000000,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-Patator
11348,20,1,1,0,0.0,0,0,0.000000,0.000000,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-Patator
11349,38,1,1,0,0.0,0,0,0.000000,0.000000,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-Patator
11350,80,1,1,0,0.0,0,0,0.000000,0.000000,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-Patator
11351,68,1,1,0,0.0,0,0,0.000000,0.000000,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-Patator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445684,13609590,22,32,2008,2745.0,640,0,91.272727,138.182137,976,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Patator
445688,13653780,22,33,2008,2745.0,640,0,91.272727,138.182137,976,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Patator
445689,13681270,22,33,2008,2745.0,640,0,91.272727,138.182137,976,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Patator
445814,13625617,22,33,2008,2745.0,640,0,91.272727,138.182137,976,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Patator
